In [ ]:
!git clone https://github.com/openai/whisper.git

!pip install torch openai-whisper

# !pip install openai-whisper
import whisper
import os

def transcribe_audio(audio_path, model_name="base"):
    # Load the Whisper model
    model = whisper.load_model(model_name)

    # Perform the transcription
    result = model.transcribe(audio_path)

    # Extract the transcribed text
    transcribed_text = result["text"]

    return transcribed_text

# Example usage
audio_file_path = "conversation.wav"  # Replace with your audio file path
output_file_path = "transcription.txt"  # Path for the output text file

# Perform transcription
transcription = transcribe_audio(audio_file_path)

# Save transcription to a text file
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(transcription)

print(f"Transcription completed and saved to {output_file_path}")

!pip install openai

%cd /content


import openai
import os
from typing import List, Tuple

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual API key

def read_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_into_chunks(text: str, chunk_size: int = 1000) -> List[str]:
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def classify_text(chunk: str, prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies text."},
            {"role": "user", "content": f"{prompt}\n\nText to classify:\n{chunk}"}
        ]
    )
    return response.choices[0].message['content'].strip()

def process_file(file_path: str, prompt: str) -> Tuple[List[str], List[str]]:
    text = read_file(file_path)
    chunks = split_into_chunks(text)

    label1_chunks = []
    label2_chunks = []

    for chunk in chunks:
        classification = classify_text(chunk, prompt)
        if "Label 1" in classification:
            label1_chunks.append(chunk)
        elif "Label 2" in classification:
            label2_chunks.append(chunk)

    return label1_chunks, label2_chunks

def save_results(label1_chunks: List[str], label2_chunks: List[str], output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, 'label1_text.txt'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(label1_chunks))

    with open(os.path.join(output_dir, 'label2_text.txt'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(label2_chunks))

# Main execution
if __name__ == "__main__":
    file_path = input("transcription.txt")
    prompt = input("please classify the female and male voice text and label them as male and female ")
    output_dir = input("opt.txt")

    label1_chunks, label2_chunks = process_file(file_path, prompt)
    save_results(label1_chunks, label2_chunks, output_dir)

    print(f"Classification complete. Results saved in {output_dir}")

!pip install openai==0.28

!pip install openai nltk

import openai
import nltk
import os

# Download the punkt tokenizer for sentence splitting
nltk.download('punkt', quiet=True)

# Set your OpenAI API key
openai.api_key = "api key"  # Replace with your actual API key

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_into_sentences(text):
    return nltk.sent_tokenize(text)

def classify_sentence(sentence):
    prompt = f"Classify the following sentence as either doctor's speech or patient's speech. Respond with only 'Patient:' followed by the sentence.\n\nSentence: {sentence}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies speech in medical conversations."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def process_transcription(input_file, output_file):
    # Read the transcription
    transcription = read_file(input_file)

    # Split into sentences
    sentences = split_into_sentences(transcription)

    # Classify each sentence
    classified_sentences = []
    for sentence in sentences:
        classification = classify_sentence(sentence)
        classified_sentences.append(classification)

    # Write the classified sentences to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        for sentence in classified_sentences:
            file.write(f"{sentence}\n")

# Main execution
if __name__ == "__main__":
    input_file = "transcription.txt"
    output_file = "opt.txt"

    process_transcription(input_file, output_file)

    print(f"Classification complete. Results saved in {output_file}")

# import re
# import nltk

# # Download the punkt tokenizer for sentence splitting
# nltk.download('punkt', quiet=True)

# # Lists of keywords and phrases often used by patients and doctors
# patient_keywords = [
#     "feel", "pain", "hurt", "worried", "concerned", "scared", "symptom",
#     "problem", "issue", "medication", "medicine", "drug", "pill",
#     "side effect", "family history", "allergy", "allergic"
# ]

# doctor_keywords = [
#     "diagnosis", "treatment", "prescription", "recommend", "suggest",
#     "test", "examination", "results", "condition", "prognosis",
#     "follow-up", "specialist", "referral", "dose", "medical history"
# ]

# question_words = ["what", "how", "when", "where", "why", "do", "is", "are", "can", "could"]

# def is_question(sentence):
#     return any(sentence.lower().startswith(word) for word in question_words) or sentence.endswith("?")

# def count_keywords(sentence, keyword_list):
#     return sum(1 for keyword in keyword_list if keyword in sentence.lower())

# def classify_sentence(sentence):
#     patient_score = count_keywords(sentence, patient_keywords)
#     doctor_score = count_keywords(sentence, doctor_keywords)

#     # Doctors tend to ask more questions
#     if is_question(sentence):
#         doctor_score += 1

#     # Patients often use first-person pronouns
#     if re.search(r'\b(i|me|my|mine)\b', sentence.lower()):
#         patient_score += 1

#     # Doctors often use second-person pronouns
#     if re.search(r'\b(you|your|yours)\b', sentence.lower()):
#         doctor_score += 1

#     if patient_score > doctor_score:
#         return "Patient: " + sentence
#     elif doctor_score > patient_score:
#         return "Doctor: " + sentence
#     else:
#         return "Unclassified: " + sentence

# def process_transcription(input_file, output_file):
#     # Read the transcription
#     with open(input_file, 'r', encoding='utf-8') as file:
#         transcription = file.read()

#     # Split into sentences
#     sentences = nltk.sent_tokenize(transcription)

#     # Classify each sentence
#     classified_sentences = [classify_sentence(sentence) for sentence in sentences]

#     # Write the classified sentences to the output file
#     with open(output_file, 'w', encoding='utf-8') as file:
#         for sentence in classified_sentences:
#             file.write(f"{sentence}\n")

# # Main execution
# if __name__ == "__main__":
#     input_file = "transcription.txt"
#     output_file = "classified_conversation.txt"

#     process_transcription(input_file, output_file)

#     print(f"Classification complete. Results saved in {output_file}")

